![Roboflow Notebooks banner](https://camo.githubusercontent.com/aec53c2b5fb6ed43d202a0ab622b58ba68a89d654fbe3abab0c0cc8bd1ff424e/68747470733a2f2f696b2e696d6167656b69742e696f2f726f626f666c6f772f6e6f7465626f6f6b732f74656d706c6174652f62616e6e657274657374322d322e706e673f696b2d73646b2d76657273696f6e3d6a6176617363726970742d312e342e33267570646174656441743d31363732393332373130313934)

# Image Classification with DINOv2

DINOv2, released by Meta Research in April 2023, implements a self-supervised method of training computer vision models.

DINOv2 was trained using 140 million images without labels. The embeddings generated by DINOv2 can be used for classification, image retrieval, segmentation, and depth estimation. With that said, Meta Research did not release heads for segmentation and depth estimation.
In this guide, we are going to build an image classifier using embeddings from DINOv2. To do so, we will:

1. Load a folder of images
2. Compute embeddings for each image
3. Save all the embeddings in a file and vector store
4. Train an SVM classifier to classify images

By the end of this notebook, we'll have a classifier trained on our dataset.

Without further ado, let's begin!

## Import Packages

First, let's import the packages we will need for this project.

In [107]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
import os
#import cv2
import json
import glob
from tqdm.notebook import tqdm

In [108]:
cwd = os.getcwd()
cwd

'/workspaces/gc_quant_trading_research'

Load folder containing the trading images

In [109]:
cwd = os.getcwd()

ROOT_DIR = os.path.join(cwd)

labels = {}
# remove the folder Now from the for loop

for folder in os.listdir(ROOT_DIR):
  if folder == 'test':
    continue 
  else:
    try:
      print(folder)
      for file in os.listdir(os.path.join(ROOT_DIR, folder)):
          if file.endswith(".png"):
              full_name = os.path.join(ROOT_DIR, folder, file)
              labels[full_name] = folder
    except:
      pass

files = labels.keys()

2D_embedding_viz.ipynb
all_embeddings.json
bear
range
# %% [markdown]
now
Dinov2_classification_gc.ipynb
2d_scatter_plot_4candles.html
2d_scatter_plot.html
bull
README.md
models
dockerfile
notebooks
.git
data
requirements.txt


In [110]:
list(files)

['/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-17 at 11.42.18.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-20 at 17.24.01.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-23 at 12.48.21.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-18 at 13.20.28.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-17 at 11.27.56.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-20 at 11.00.31.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-20 at 17.13.05.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-20 at 17.25.53.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-10-03 at 10.56.40.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-17 at 12.06.59.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-26 at 12.36.01.png',
 '/workspaces/gc_quant_trading_research/bear/Screenshot 2024-09-2

In [111]:
# prompt: get data from dictionary files

values = [labels[key] for key in files]

## Load the Model and Compute Embeddings

To train our classifier, we need:

1. The embeddings associated with each image in our dataset, and;
2. The labels associated with each image.

To calculate embeddings, we'll use DINOv2. Below, we load the smallest DINOv2 weights and define functions that will load and compute embeddings for every image in a specified list.

We store all of our vectors in a dictionary that is saved to disk so we can reference them again if needed. Note that in production environments one may opt for using another data structure such as a vector embedding database (i.e. faiss) for storing embeddings.

In [177]:
MODEL = "dinov2_vitb14"
EMBEDDING_SIZE = 768 # 768 for vitb32, 384 for vits32

dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", MODEL)

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

dinov2_vits14.to(device)

transform_image = T.Compose([T.ToTensor(),
                             T.Resize((70, 210)),
                             #T.CenterCrop(224),
                             T.Normalize([0.5], [0.5])])

Using cache found in /home/codespace/.cache/torch/hub/facebookresearch_dinov2_main


In [178]:
def load_image(img: str) -> torch.Tensor:
    """
    Load an image and return a tensor that can be used as an input to DINOv2.
    """
    img = Image.open(img)

    transformed_img = transform_image(img)[:3].unsqueeze(0)

    return transformed_img

def compute_embeddings(files: list) -> dict:
    """
    Create an index that contains all of the images in the specified list of files.
    """
    all_embeddings = {}

    with torch.no_grad():
      for i, file in enumerate(files):
        embeddings = dinov2_vits14(load_image(file).to(device))

        all_embeddings[file] = np.array(embeddings[0].cpu().numpy()).reshape(1, -1).tolist()

    with open("all_embeddings.json", "w") as f:
        f.write(json.dumps(all_embeddings))

    return all_embeddings

## Compute Embeddings

The code below computes the embeddings for all the images in our dataset. This step will take a few minutes for the MIT Indoor Scene Recognition dataset. There are over 10,000 images in the training set that we need to pass through DINOv2.

In [114]:
embeddings = compute_embeddings(files)

In [115]:
#get 1st dictionary key value
key = list(embeddings.keys())[0]
embeddings[key]

[[-0.16968728601932526,
  0.7602282762527466,
  -3.4309990406036377,
  0.056836407631635666,
  -0.7120945453643799,
  -1.5284806489944458,
  1.7471694946289062,
  -1.8829401731491089,
  0.6725326180458069,
  2.9598143100738525,
  -0.007120930589735508,
  -0.07384225726127625,
  -1.4376097917556763,
  -0.046827156096696854,
  3.0230114459991455,
  -2.3156397342681885,
  -3.47939133644104,
  -2.975426197052002,
  -0.2799299359321594,
  -1.9825143814086914,
  0.15182803571224213,
  3.253664255142212,
  0.12241896986961365,
  -1.2440340518951416,
  2.3042452335357666,
  -0.581209659576416,
  0.8111644983291626,
  2.960322618484497,
  2.8846969604492188,
  -3.995251178741455,
  -1.4678741693496704,
  1.5077781677246094,
  0.7302536964416504,
  -1.0852742195129395,
  -1.1904247999191284,
  -0.7251029014587402,
  3.9587790966033936,
  -3.364917755126953,
  3.127927780151367,
  1.5094330310821533,
  -0.1708095669746399,
  -1.1182386875152588,
  -4.154793739318848,
  2.47318959236145,
  -0.6532

In [116]:
# Check the shape of embedding_list before reshaping
embedding_list = list(embeddings.values())
embedding_array = np.array(embedding_list)
print(f"Original shape: {embedding_array.shape}")
print(f"Total number of elements: {embedding_array.size}")

# Attempt to reshape
reshaped_array = embedding_array.reshape(-1, EMBEDDING_SIZE)
print(f"Reshaped array shape: {reshaped_array.shape}")

Original shape: (1464, 1, 384)
Total number of elements: 562176
Reshaped array shape: (1464, 384)


## Train a Classification Model

The embeddings we have computed can be used as an input in a classification model. For this guide, we will be using SVM, a linear classification model.

Below, we make lists of both all of the embeddings we have computed and their associated labels. We then fit our model using those lists.

In [171]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

y = [labels[file] for file in files]
embedding_list = list(embeddings.values())
embedding_arr = np.array(embedding_list).reshape(-1, EMBEDDING_SIZE)
N_COMPONENTS = 50
N_ESTIMATORS = 6
MAX_DEPTH = 104
R_STATE = 5

#fit a svm model
def fit_svm_pca(embedding_list, y):
    clf = svm.SVC(gamma='scale')
    # Convert embedding_list to a 2D array
    embedding_array = np.array(embedding_list).reshape(len(embedding_list), -1)
    
    # Apply PCA to reduce dimensions to x principal components
    pca = PCA(n_components=N_COMPONENTS, random_state=R_STATE)
    embedding_list = pca.fit_transform(embedding_array)
    
    clf.fit(embedding_list, y)
    return clf, pca

clf_svm_pca, pca = fit_svm_pca(embedding_list, y)


#fit a svm model with pca
def fit_svm(embedding_list, y):
    clf = svm.SVC(gamma='scale')
    clf.fit(np.array(embedding_list).reshape(-1, EMBEDDING_SIZE), y)
    return clf

clf_svm = fit_svm(embedding_list, y)


#fit a random forest model
def fit_rf(embedding_list, y):
    rf = RandomForestClassifier(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=R_STATE) #70
    rf.fit(np.array(embedding_list).reshape(-1, EMBEDDING_SIZE), y)
    return rf

clf_rf = fit_rf(embedding_list, y)
#save this rf model


#fit a random forest model with pca
def fit_rf_pca(embedding_list, y):
    rf = RandomForestClassifier(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=R_STATE) #70

    # Convert embedding_list to a 2D array
    embedding_array = np.array(embedding_list).reshape(len(embedding_list), -1)
    
    # Apply PCA to reduce dimensions to x principal components
    pca = PCA(n_components=N_COMPONENTS, random_state=R_STATE)
    embedding_list = pca.fit_transform(embedding_array)

    rf.fit(embedding_list, y)
    return rf, pca

clf_rf_pca, rf_pca = fit_rf_pca(embedding_list, y)




In [172]:
import joblib
save_model = "no"
if save_model == "yes":

    joblib.dump(clf_rf_pca, 'models/rf_pca/best_rf_model_oct1st.pkl')
    joblib.dump(rf_pca, 'models/rf_pca/best_pca_model.pkl')

## Classify an Image

We now have a classifier we can use to classify images!

Change the `input_file` value below to the path of a file in the `valid` or `test` directories in the image dataset with which we have been working.

Then, run the cell to classify the image.

In [173]:
#any file in the folder title Now
TEST_FOLDER = "bear"
input_files = glob.glob(f'test/{TEST_FOLDER}/*.png')

for file in glob.glob(f'test/bull/*.png'):
    input_files.append(file)


In [174]:
input_files

['test/bear/Screenshot 2024-10-03 at 11.56.33.png',
 'test/bear/Screenshot 2024-10-03 at 11.55.09.png',
 'test/bear/Screenshot 2024-10-03 at 11.56.21.png',
 'test/bear/Screenshot 2024-10-03 at 11.56.10.png',
 'test/bear/Screenshot 2024-10-03 at 11.55.32.png',
 'test/bear/Screenshot 2024-10-03 at 11.56.01.png',
 'test/bear/Screenshot 2024-10-03 at 11.56.45.png',
 'test/bear/Screenshot 2024-10-03 at 11.55.39.png',
 'test/bear/Screenshot 2024-10-03 at 11.54.34.png',
 'test/bear/Screenshot 2024-10-03 at 11.55.48.png',
 'test/bear/Screenshot 2024-10-03 at 11.54.53.png',
 'test/bear/Screenshot 2024-10-03 at 11.57.03.png',
 'test/bull/Screenshot 2024-10-03 at 11.52.05.png',
 'test/bull/Screenshot 2024-10-03 at 11.52.44.png',
 'test/bull/Screenshot 2024-10-03 at 11.51.25.png',
 'test/bull/Screenshot 2024-10-03 at 11.53.52.png',
 'test/bull/Screenshot 2024-10-03 at 11.52.17.png',
 'test/bull/Screenshot 2024-10-03 at 11.53.25.png',
 'test/bull/Screenshot 2024-10-03 at 11.50.06.png',
 'test/bull/

In [175]:
# Initialize an empty DataFrame
import pandas as pd

predictions_df = pd.DataFrame(columns=['image_file', 'svm_prediction', 'svm_pca_prediction', 'rf_prediction', 
                                       'rf_pca_prediction', 'best_rf_pca_prediction', 'best_rf_prediction', 'best_svm_pca_prediction'])



for input_file in input_files:
    new_image = load_image(input_file)

    with torch.no_grad():
        new_embedding = dinov2_vits14(new_image.to(device))

        # Convert embedding to numpy array and reshape
        new_embedding_array = np.array(new_embedding[0].cpu()).reshape(1, -1)
        new_embedding_pca = pca.transform(new_embedding_array)
        
        # Generate with newly trained model the predictions
        svm_pca_prediction = clf_svm_pca.predict(new_embedding_pca)
        svm_prediction = clf_svm.predict(new_embedding_array)
        rf_prediction = clf_rf.predict(new_embedding_array)
        rf_pca_prediction = clf_rf_pca.predict(new_embedding_pca)

        def load_best_models():
            best_clf_rf_pca = joblib.load('models/rf_pca/best_rf_model_oct1st.pkl')
            best_rf_pca = joblib.load('models/rf_pca/best_pca_model.pkl')

            best_clf_svm_pca = joblib.load('models/svm_pca/best_svm_model_oct1st.pkl')
            best_svm_pca = joblib.load('models/svm_pca/best_pca_model.pkl')


            best_clf_rf = joblib.load('models/rf/best_rf_model_30sept.pkl')
            
            return best_clf_rf_pca, best_clf_rf, best_rf_pca, best_svm_pca, best_clf_svm_pca

        best_clf_rf_pca, best_clf_rf, best_rf_pca, best_svm_pca, best_clf_svm_pca = load_best_models()

        # Generate predictions with saved best models
        best_embedding_rf_pca = best_rf_pca.transform(new_embedding_array)
        best_rf_pca_prediction = best_clf_rf_pca.predict(best_embedding_rf_pca)
        best_rf_prediction = best_clf_rf.predict(new_embedding_array)

        best_embedding_svm_pca = best_svm_pca.transform(new_embedding_array)
        #print(best_embedding_svm_pca)
        best_svm_pca_prediction = best_clf_svm_pca.predict(best_embedding_svm_pca)

        # Add the predictions to the DataFrame using loc
        predictions_df.loc[len(predictions_df)] = [input_file, svm_prediction[0],svm_pca_prediction[0],
                                                    rf_prediction[0], rf_pca_prediction[0], best_rf_pca_prediction[0],
                                                    best_rf_prediction[0], best_svm_pca_prediction[0]]


In [176]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Extract true labels from the file paths
predictions_df['true_label'] = predictions_df['image_file'].apply(lambda x: 'bull' if 'bull' in x else 'bear')


# Compute accuracy for SVM and RF predictions
svm_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['svm_prediction'])
svm_pca_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['svm_pca_prediction'])
rf_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['rf_prediction'])
rf_pca_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['rf_pca_prediction'])
best_rf_pca_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['best_rf_pca_prediction'])
best_rf_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['best_rf_prediction'])
best_svm_pca_accuracy = accuracy_score(predictions_df['true_label'], predictions_df['best_svm_pca_prediction'])

# Generate confusion matrices
svm_conf_matrix = confusion_matrix(predictions_df['true_label'], predictions_df['svm_prediction'])
svm_pca_conf_matrix = confusion_matrix(predictions_df['true_label'], predictions_df['svm_pca_prediction'])
rf_conf_matrix = confusion_matrix(predictions_df['true_label'], predictions_df['rf_prediction'])
rf_pca_conf_matrix = confusion_matrix(predictions_df['true_label'], predictions_df['rf_pca_prediction'])
best_rf_pca_conf_matrix = confusion_matrix(predictions_df['true_label'], predictions_df['best_rf_pca_prediction'])
best_rf_conf_matrix = confusion_matrix(predictions_df['true_label'], predictions_df['best_rf_prediction'])
best_svm_pca_conf_matrix = confusion_matrix(predictions_df['true_label'], predictions_df['best_svm_pca_prediction'])

print('ALL')
print('--------------------------------------------------')
print(f"SVM Accuracy: {round(svm_accuracy, 2)}")
print(f"RF Accuracy: {round(rf_accuracy,2)}")
print(f"SVM PCA Accuracy: {round(svm_pca_accuracy,2)}")
print(f"RF PCA Accuracy: {round(rf_pca_accuracy,2)}")
print('--------------------------------------------------')
print(f"Best RF PCA Accuracy: {round(best_rf_pca_accuracy,2)}")
print(f"Best SVM PCA Accuracy: {round(best_svm_pca_accuracy,2)}")
print(f"Best RF Accuracy: {round(best_rf_accuracy,2)}")


predictions_df

ALL
--------------------------------------------------
SVM Accuracy: 0.44
RF Accuracy: 0.4
SVM PCA Accuracy: 0.4
RF PCA Accuracy: 0.44
--------------------------------------------------
Best RF PCA Accuracy: 0.44
Best SVM PCA Accuracy: 0.48
Best RF Accuracy: 0.36


,image_file,svm_prediction,svm_pca_prediction,rf_prediction,rf_pca_prediction,best_rf_pca_prediction,best_rf_prediction,best_svm_pca_prediction,true_label
0,test/bear/Screenshot 2024-10-03 at 11.56.33.png,bear,bear,bull,bear,bear,bull,bull,bear
1,test/bear/Screenshot 2024-10-03 at 11.55.09.png,bear,bear,bear,bull,bear,bear,bull,bear
2,test/bear/Screenshot 2024-10-03 at 11.56.21.png,bear,bear,bear,bull,bull,bear,bear,bear
3,test/bear/Screenshot 2024-10-03 at 11.56.10.png,bull,bear,bear,bear,bear,bear,bull,bear
4,test/bear/Screenshot 2024-10-03 at 11.55.32.png,bear,bear,bull,bull,bear,bear,bear,bear
5,test/bear/Screenshot 2024-10-03 at 11.56.01.png,bull,bull,bear,bear,bear,bull,bull,bear
6,test/bear/Screenshot 2024-10-03 at 11.56.45.png,bear,bear,bull,bear,bull,bull,bear,bear
7,test/bear/Screenshot 2024-10-03 at 11.55.39.png,bear,bear,bull,bear,bull,bull,bear,bear
8,test/bear/Screenshot 2024-10-03 at 11.54.34.png,bull,bull,bear,bull,bull,bull,bull,bear
9,test/bear/Screenshot 2024-10-03 at 11.55.48.png,bull,bull,bear,range,bull,bear,bull,bear
